<a href="https://colab.research.google.com/github/A01720548/TC2008B.4-ActividadIntegradora/blob/main/ActividadIntegradoraFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esteban de la Maza A01720548

In [ ]:
!pip3 install mesa

In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `Multigrid` que nos deja tener varios agentes en una celda
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import signal

import math

In [ ]:
# VARIABLES GLOBALES
stackedBoxes = 0
numberOfBoxes = 0
numberOfStacks = 0
residue = 0
numStack = 0
boxesDepositedInCurrentStack = 0


def startCalculations(K):
  # Esta funcion realiza una serie de calculos para imprimir y mostrar informacion relevante al usuario
  global numberOfBoxes
  global numberOfStacks
  global residue
  numberOfBoxes = K
  numberOfStacks = math.floor(K/5)
  residue = K % 5
  print('number of boxes', numberOfBoxes)
  print('number of stacks', numberOfStacks)
  print('residue', residue)




def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
            grid[x][y] = agent.height
    return grid

class Robot(Agent):
  # Clase tipo robot, se inicializa sin tener caja y con 6 de altura para diferenciarlo de las cajas y stacks
    def __init__(self, unique_id, model, height):
      super().__init__(unique_id, model)
      self.height = height
      self.hasBox = 0
      self.box = -1


    def depositBox(self):
      global boxesDepositedInCurrentStack
      global stackedBoxes
      global numStack
      b = Box(self.pos, self, boxesDepositedInCurrentStack + 1)
      b.stacked = True
      self.model.grid.place_agent(b, self.pos)
      boxesDepositedInCurrentStack +=1
      self.hasBox = 2
      stackedBoxes +=1
      if boxesDepositedInCurrentStack == 5:
        numStack +=1
        boxesDepositedInCurrentStack = 0


    def canIMoveThere(self, pos):
      neighbours = self.model.grid.get_neighbors(
          self.pos,
          moore=False,
          include_center=False)
      for neighbour in neighbours:
        if (isinstance(neighbour, Robot) and neighbour.pos == pos):
          return False
      return True         

    def findBox(self):
      neighbours = self.model.grid.get_neighbors(
          self.pos,
          moore=False,
          include_center=True)
      for neighbour in neighbours:
        if (isinstance(neighbour, Box)):
          if (neighbour.height) == 1 and neighbour.stacked == False:
            return neighbour
      return -1 
          
    def moveFromStack(self) :
      x, y = self.pos
      if x != self.model.width/2 and self.canIMoveThere((x+1,y)):
        self.model.grid.move_agent(self, (x+1,y))
      if y != self.model.height/2 and self.canIMoveThere((x,y+1)):
        self.model.grid.move_agent(self, (x,y+1))
      if x==self.model.width/2 and y==self.model.height/2:
        self.hasBox = 0  

    def moveToStack(self):
      global numStack
      x, y = self.pos
      if x != 0 and self.canIMoveThere((x-1,y)):
        self.model.grid.move_agent(self, (x-1,y))
      if y < numStack and self.canIMoveThere((x,y+1)):
        self.model.grid.move_agent(self, (x,y+1))
      if y > numStack and self.canIMoveThere((x,y-1)):
        self.model.grid.move_agent(self, (x,y-1))
      if x==0 and y==numStack:
        self.depositBox()


    def random_move(self):
      next_moves = self.model.grid.get_neighborhood(self.pos, False, True)
      next_move = self.random.choice(next_moves)
      # Realizar el movimiento
      xNext, yNext = next_move
      x, y = self.pos

      if self.canIMoveThere(next_move):
        if (x == 0 and xNext == self.model.height - 1):
          return
        if (y == 0 and yNext == self.model.width - 1):
          return
        if (x == self.model.height - 1 and xNext == 0):
          return
        if (y == self.model.width - 1 and yNext == 0):
          return    
        self.model.grid.move_agent(self, next_move)



    def step(self):
      if self.hasBox == 0:
        self.box = self.findBox()
        if self.box != -1:             
          self.hasBox = 1
          self.model.grid.remove_agent(self.box)
        

    def advance(self):
      if self.hasBox == 1:
        self.moveToStack()
      elif self.hasBox == 2:
        self.moveFromStack()
      else:
        self.random_move()



class Box(Agent):

    def __init__(self, unique_id, model, height):
      super().__init__(unique_id, model)
      self.height = height
      self.stacked = False
      

class RobotModel(Model):
    '''
    Define el modelo donde se encuentran los agentes de aspiradoras y piso. Se crea el MultiGrid y se agregan agentes
    a las casillas. Si es tipo vaccuum su posicion inicial es (1,1) y se agrega un 0 para identificarlo como Vaccuum
    Si es tipo Floor se agrega a cada casilla y se le da un 1 en su Unique ID para identificarlo
    '''
    def __init__(self, width, height, K):
      self.grid = MultiGrid(width, height, False)
      self.schedule = SimultaneousActivation(self)
      self.numBoxes = K
      self.width = width
      self.height = height
      global numberOfBoxes
      startCalculations(K)
      gridSize = width*height
      # print(total)
      arrayPositions = np.zeros((width,height), int)
      x = 0
      while x < K:
        pos = np.random.randint(0,width-1)
        pos2 = np.random.randint(0,height-1)
        if arrayPositions[pos][pos2] == 0:
          arrayPositions[pos][pos2] = 1
          x+=1

      y = 0
      while y < 5:
        pos = np.random.randint(0,width-1)
        pos2 = np.random.randint(0,height-1)
        if arrayPositions[pos][pos2] == 0:
          arrayPositions[pos][pos2] = 2
          y+=1
      print(arrayPositions)
      for (content, x, y) in self.grid.coord_iter():
        if arrayPositions[x][y] == 2:
          a = Robot((x, y), self, 6)
          self.grid.place_agent(a, (x,y)) 
          self.schedule.add(a)
        if arrayPositions[x][y] == 1:
          b = Box((x, y), self, 1)
          self.grid.place_agent(b, (x,y))
          self.schedule.add(b)

      
      # Aquí definimos con colector para obtener el grid completo.
      self.datacollector = DataCollector(
          model_reporters={"Grid": get_grid})
    
    def allStacked(self):
      self.percentageStacked = stackedBoxes / self.numBoxes
      if self.percentageStacked == 1 :
        return 1
      else:
        return self.percentageStacked



    def step(self):
      '''
      En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
      '''
      self.datacollector.collect(self)
      self.schedule.step()

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 10
# Definimos el número de cajas
K = 25


# Max RunTime
start_time = time.time()
maxTime = .01

frameCounter = 0

model = RobotModel(GRID_SIZE, GRID_SIZE, K)
# and model.allClean() != 1
while(time.time() - start_time) < maxTime and model.allStacked() != 1:
  frameCounter+=1
  model.step()

frameCounter+=1
model.step()


# Tiempo de Ejecucion
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

# Porcentaje de Celas Limpias despues del Termino de la Simulacion
print('Porcentaje de Cajas Stacked: ', str(model.allStacked() * 100), '%')

# Numero de Movimientos Realizados por los Agentes
print('Numero de Movimientos Realizados: ', frameCounter)


number of boxes 25
number of stacks 5
residue 0
[[1 0 1 0 0 0 1 0 1 0]
 [0 0 1 0 1 1 0 1 0 0]
 [0 0 0 1 0 0 0 1 0 0]
 [0 0 1 0 0 2 2 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [1 0 1 2 1 0 1 2 1 0]
 [0 0 0 0 2 1 1 1 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
Tiempo de ejecución: 0:00:00.010363
Porcentaje de Cajas Stacked:  40.0 %
Numero de Movimientos Realizados:  51


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(all_grid.iloc[0][0])

[[1. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 6. 6. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 6. 1. 0. 1. 6. 1. 0.]
 [0. 0. 0. 0. 6. 1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.Greens)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=frameCounter)

In [ ]:

anim